In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.models import Sequential

import os

In [2]:
! rm -r ./data
! mkdir ./data
! tar -xf ./file.tgz -C ./data

In [3]:
batch_size = 32
img_height = 180
img_width = 180
seed = 42

In [4]:
image_size = (img_height, img_width)

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "./data",
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
)

Found 3000 files belonging to 2 classes.
Using 2400 files for training.
Using 600 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['cats', 'dogs']


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,img_width,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

/Users/maxkleinegger/anaconda3/lib/python3.10/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
num_classes = len(class_names)

model = Sequential([
  # data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - accuracy: 0.5516 - loss: 0.6769 - val_accuracy: 0.6200 - val_loss: 0.6443
Epoch 2/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 130ms/step - accuracy: 0.6785 - loss: 0.5805 - val_accuracy: 0.7150 - val_loss: 0.5775
Epoch 3/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 143ms/step - accuracy: 0.7806 - loss: 0.4721 - val_accuracy: 0.7267 - val_loss: 0.5591
Epoch 4/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 142ms/step - accuracy: 0.8539 - loss: 0.3545 - val_accuracy: 0.6983 - val_loss: 0.6769
Epoch 5/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 142ms/step - accuracy: 0.8617 - loss: 0.3135 - val_accuracy: 0.7233 - val_loss: 0.7360
Epoch 6/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 140ms/step - accuracy: 0.9344 - loss: 0.1926 - val_accuracy: 0.7067 - val_loss: 0.8569
Epoch 7/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 132ms/step - accuracy: 0.9506 - loss: 0.1441 - val_accuracy: 0.7283 - val_loss: 0.9274
Epoch 8/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - accuracy: 0.9608 - loss: 0.1038 - val_accu

In [12]:
model.save_weights('./model.weights.h5')

In [16]:
data_url = "file://" + os.path.expanduser("~/Downloads/test_cat.jpg")
data_path = tf.keras.utils.get_file('test_cat', origin=data_url)

img = tf.keras.utils.load_img(
    data_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
This image most likely belongs to cats with a 99.43 percent confidence.
